# Install requirements

In [1]:
!pip install sentence-transformers
!pip install qdrant-client
!pip install langchain
!pip install pypdf
!pip install protobuf
!pip install python-dotenv


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Set Globals
Set the global variables like embedding model here!

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

EMBEDDING_MODEL = "intfloat/multilingual-e5-small"
RERANK_MODEL = "svalabs/cross-electra-ms-marco-german-uncased"#"cross-encoder/msmarco-MiniLM-L6-en-de-v1" #"intfloat/multilingual-e5-small"
QUERY = "Wer ist Alaeddin?"
FETCH_K = 100
RERANK_K = 5
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

AttributeError: module 'dotenv' has no attribute 'load'

# Initialize Vectorstore
Here we initialize the vectorstore with a example german document "Alaeddin"

In [ ]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores.qdrant import Qdrant
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
doc_loader = PyPDFDirectoryLoader("./data/")
documents = doc_loader.load_and_split(text_splitter=CharacterTextSplitter(chunk_size=100, chunk_overlap=50))

embedding_model = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
vector_store = Qdrant.from_documents(documents=documents, embedding=embedding_model, location=":memory:", collection_name="rerank")

In [ ]:
vector_store.max_marginal_relevance_search("Wer ist Alaeddin?", k=4, fetch_k=20)

[Document(page_content='Alaeddin\tund\tdie\tWunderlampe\nM\nustafa\twar\tder\tName\teines\tSchneiders,\tder\tin\teiner\tsehr\treichen\tund\tgroßen\nHauptstadt\tChinas\tlebte.\tDieser\tMustafa\twar\tsehr\tarm,\tund\tseine\tArbeit\twarf\nkaum\tso\tviel\tab,\tdaß\ter,\tseine\tFrau\tund\tein\tSohn\tdavon\tleben\tkonnten.\nDie\tErziehung\tdieses\tSohnes,\twelcher\tAlaeddin\thieß,\twar\tsehr\tvernachlässigt\nworden,\tso\tdaß\ter\tallerhand\tlasterhafte\tNeigungen\tangenommen\thatte.\tEr\twar\nboshaft,\thalsstarrig\tund\tungehorsam\tgegen\tVater\tund\tMutter.\tKaum\twar\ter\tein\nwenig\therangewachsen,\tso\tkonnten\tihn\tseine\tEltern\tnicht\tmehr\tim\tHause\nzurückhalten.\tEr\tging\tschon\tam\tfrühen\tMorgen\taus\tund\ttat\tden\tganzen\tTag\nnichts,\tals\tauf\tden\tStraßen\tund\töffentlichen\tPlätzen\tmit\tkleinen\tTagdieben\nspielen.\nAls\ter\tein\tHandwerk\terlernen\tsollte,\tnahm\tihn\tsein\tVater\tin\tseine\tBude\tund\nfing\tan,\tihn\tin\tder\tHandhabung\tder\tNadel\tzu\tunterrichten.\tA

In [ ]:
vector_store.similarity_search("Wer ist Alaeddin?", k=4)

[Document(page_content='Alaeddin\tund\tdie\tWunderlampe\nM\nustafa\twar\tder\tName\teines\tSchneiders,\tder\tin\teiner\tsehr\treichen\tund\tgroßen\nHauptstadt\tChinas\tlebte.\tDieser\tMustafa\twar\tsehr\tarm,\tund\tseine\tArbeit\twarf\nkaum\tso\tviel\tab,\tdaß\ter,\tseine\tFrau\tund\tein\tSohn\tdavon\tleben\tkonnten.\nDie\tErziehung\tdieses\tSohnes,\twelcher\tAlaeddin\thieß,\twar\tsehr\tvernachlässigt\nworden,\tso\tdaß\ter\tallerhand\tlasterhafte\tNeigungen\tangenommen\thatte.\tEr\twar\nboshaft,\thalsstarrig\tund\tungehorsam\tgegen\tVater\tund\tMutter.\tKaum\twar\ter\tein\nwenig\therangewachsen,\tso\tkonnten\tihn\tseine\tEltern\tnicht\tmehr\tim\tHause\nzurückhalten.\tEr\tging\tschon\tam\tfrühen\tMorgen\taus\tund\ttat\tden\tganzen\tTag\nnichts,\tals\tauf\tden\tStraßen\tund\töffentlichen\tPlätzen\tmit\tkleinen\tTagdieben\nspielen.\nAls\ter\tein\tHandwerk\terlernen\tsollte,\tnahm\tihn\tsein\tVater\tin\tseine\tBude\tund\nfing\tan,\tihn\tin\tder\tHandhabung\tder\tNadel\tzu\tunterrichten.\tA

# Simple bm25 Keywoard search addon

In [ ]:
!pip install rank_bm25


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents=documents)

# Reranking
Resource 1: https://www.sbert.net/examples/applications/retrieve_rerank/README.html
## Cross-Encoder
Starting rerankin with an additional Cross-Encoder as supported by SentenceTransformers (https://www.sbert.net/examples/applications/cross-encoder/README.html)

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

cross_model = CrossEncoder(model_name=RERANK_MODEL)

In [ ]:
# Retrieve top 100 passages with MMR and Cos Similarity each
cos_documents = vector_store.similarity_search(QUERY, k=FETCH_K)

mmr_documents = vector_store.max_marginal_relevance_search(QUERY, k=FETCH_K, fetch_k=FETCH_K*2)

# Retrieve top 100 passages with BM25 Keyword search
bm25_documents = bm25_retriever.get_relevant_documents(QUERY)
len(bm25_documents)

4

In [ ]:
# Add bm25 docs to cos and mmr docs
cos_documents = cos_documents + bm25_documents
mmr_documents = mmr_documents + bm25_documents

In [ ]:
# get cross encoder score for query to passage
cos_combinations = [[QUERY, document.page_content] for document in cos_documents]
mmr_combinations = [[QUERY, document.page_content] for document in mmr_documents]

cos_similarity_scores = cross_model.predict(cos_combinations, show_progress_bar=True)
mmr_similarity_scores = cross_model.predict(mmr_combinations, show_progress_bar=True)

Batches: 100%|██████████| 4/4 [01:03<00:00, 15.99s/it]


In [ ]:
import numpy as np

# Sort scores
cos_similarity_scores_sorted = np.argsort(cos_similarity_scores)
mmr_similarity_scores_sorted = np.argsort(mmr_similarity_scores)

In [ ]:
# Print scores for cos sim
print("Query: ", QUERY)
for idx in cos_similarity_scores_sorted:
    print("{:.2f}\t{}".format(cos_similarity_scores[idx], idx))

Query:  Wer ist Alaeddin?
-11.84	93
-11.84	99
-11.84	98
-11.84	97
-11.84	96
-11.84	95
-11.84	94
-11.84	89
-11.84	92
-11.84	91
-11.84	90
-9.23	78
-6.70	81
-6.26	86
-6.23	88
-6.14	84
-5.52	79
-5.24	80
-4.20	85
-2.88	76
-1.58	82
-1.40	87
-1.05	67
-0.99	69
-0.74	74
-0.58	83
-0.38	68
-0.34	22
-0.06	101
-0.06	66
0.08	62
0.33	55
0.43	14
0.48	64
0.57	37
0.59	43
0.67	45
0.73	71
0.75	33
0.78	52
0.79	51
0.85	46
0.86	61
0.89	73
0.89	103
0.98	1
1.05	63
1.14	2
1.14	48
1.21	77
1.27	4
1.29	31
1.29	38
1.32	35
1.33	58
1.37	23
1.41	28
1.51	27
1.56	5
1.57	72
1.65	75
1.66	6
1.70	21
1.74	29
1.75	59
1.79	25
1.81	12
1.86	8
1.86	44
1.93	57
2.03	7
2.08	24
2.20	16
2.21	50
2.30	100
2.30	20
2.37	54
2.39	102
2.39	70
2.40	9
2.48	32
2.50	42
2.65	41
2.67	36
2.88	10
2.92	26
2.97	15
3.00	39
3.02	40
3.06	3
3.06	53
3.11	19
3.19	47
3.49	30
3.52	17
3.65	60
3.84	13
3.90	18
3.97	49
4.18	56
4.23	34
4.66	65
4.80	11
8.20	0


In [ ]:
# Print scores for mmr sim
print("Query: ", QUERY)
for idx in mmr_similarity_scores_sorted:
    print("{:.2f}\t{}".format(mmr_similarity_scores[idx], idx))

Query:  Wer ist Alaeddin?
-11.84	99
-11.84	1
-11.84	97
-11.84	96
-11.84	98
-10.91	95
-10.82	92
-10.34	94
-9.71	93
-9.61	90
-9.23	81
-8.98	4
-6.70	86
-6.26	89
-6.23	91
-6.14	76
-5.52	88
-5.24	84
-4.20	71
-2.88	73
-1.58	64
-1.40	87
-1.05	59
-0.99	38
-0.74	69
-0.58	5
-0.38	56
-0.34	42
-0.06	101
-0.06	58
0.08	34
0.33	82
0.43	9
0.48	70
0.57	29
0.59	57
0.67	61
0.73	74
0.75	45
0.78	77
0.79	68
0.85	53
0.86	47
0.89	83
0.89	103
0.98	3
1.05	72
1.14	10
1.14	66
1.21	85
1.27	23
1.29	52
1.29	46
1.32	15
1.33	65
1.37	30
1.41	28
1.51	36
1.56	7
1.57	79
1.65	50
1.66	6
1.70	21
1.74	44
1.75	43
1.79	18
1.81	31
1.86	24
1.86	41
1.93	33
2.03	11
2.08	62
2.20	54
2.21	78
2.30	100
2.30	17
2.37	39
2.39	75
2.39	102
2.40	13
2.48	37
2.50	32
2.65	80
2.67	27
2.88	14
2.92	16
2.97	26
3.00	48
3.02	8
3.06	2
3.06	60
3.11	20
3.19	63
3.49	55
3.52	40
3.65	22
3.84	25
3.90	19
3.97	49
4.18	67
4.23	51
4.66	35
4.80	12
8.20	0


In [ ]:
cos_combinations[0]

['Wer ist Alaeddin?',
 'Alaeddin\tund\tdie\tWunderlampe\nM\nustafa\twar\tder\tName\teines\tSchneiders,\tder\tin\teiner\tsehr\treichen\tund\tgroßen\nHauptstadt\tChinas\tlebte.\tDieser\tMustafa\twar\tsehr\tarm,\tund\tseine\tArbeit\twarf\nkaum\tso\tviel\tab,\tdaß\ter,\tseine\tFrau\tund\tein\tSohn\tdavon\tleben\tkonnten.\nDie\tErziehung\tdieses\tSohnes,\twelcher\tAlaeddin\thieß,\twar\tsehr\tvernachlässigt\nworden,\tso\tdaß\ter\tallerhand\tlasterhafte\tNeigungen\tangenommen\thatte.\tEr\twar\nboshaft,\thalsstarrig\tund\tungehorsam\tgegen\tVater\tund\tMutter.\tKaum\twar\ter\tein\nwenig\therangewachsen,\tso\tkonnten\tihn\tseine\tEltern\tnicht\tmehr\tim\tHause\nzurückhalten.\tEr\tging\tschon\tam\tfrühen\tMorgen\taus\tund\ttat\tden\tganzen\tTag\nnichts,\tals\tauf\tden\tStraßen\tund\töffentlichen\tPlätzen\tmit\tkleinen\tTagdieben\nspielen.\nAls\ter\tein\tHandwerk\terlernen\tsollte,\tnahm\tihn\tsein\tVater\tin\tseine\tBude\tund\nfing\tan,\tihn\tin\tder\tHandhabung\tder\tNadel\tzu\tunterrichten.\tA

In [ ]:
cos_documents[0].page_content

'Alaeddin\tund\tdie\tWunderlampe\nM\nustafa\twar\tder\tName\teines\tSchneiders,\tder\tin\teiner\tsehr\treichen\tund\tgroßen\nHauptstadt\tChinas\tlebte.\tDieser\tMustafa\twar\tsehr\tarm,\tund\tseine\tArbeit\twarf\nkaum\tso\tviel\tab,\tdaß\ter,\tseine\tFrau\tund\tein\tSohn\tdavon\tleben\tkonnten.\nDie\tErziehung\tdieses\tSohnes,\twelcher\tAlaeddin\thieß,\twar\tsehr\tvernachlässigt\nworden,\tso\tdaß\ter\tallerhand\tlasterhafte\tNeigungen\tangenommen\thatte.\tEr\twar\nboshaft,\thalsstarrig\tund\tungehorsam\tgegen\tVater\tund\tMutter.\tKaum\twar\ter\tein\nwenig\therangewachsen,\tso\tkonnten\tihn\tseine\tEltern\tnicht\tmehr\tim\tHause\nzurückhalten.\tEr\tging\tschon\tam\tfrühen\tMorgen\taus\tund\ttat\tden\tganzen\tTag\nnichts,\tals\tauf\tden\tStraßen\tund\töffentlichen\tPlätzen\tmit\tkleinen\tTagdieben\nspielen.\nAls\ter\tein\tHandwerk\terlernen\tsollte,\tnahm\tihn\tsein\tVater\tin\tseine\tBude\tund\nfing\tan,\tihn\tin\tder\tHandhabung\tder\tNadel\tzu\tunterrichten.\tAllein\tweder\tgute\nWor

In [ ]:
mmr_documents[0].page_content

'Alaeddin\tund\tdie\tWunderlampe\nM\nustafa\twar\tder\tName\teines\tSchneiders,\tder\tin\teiner\tsehr\treichen\tund\tgroßen\nHauptstadt\tChinas\tlebte.\tDieser\tMustafa\twar\tsehr\tarm,\tund\tseine\tArbeit\twarf\nkaum\tso\tviel\tab,\tdaß\ter,\tseine\tFrau\tund\tein\tSohn\tdavon\tleben\tkonnten.\nDie\tErziehung\tdieses\tSohnes,\twelcher\tAlaeddin\thieß,\twar\tsehr\tvernachlässigt\nworden,\tso\tdaß\ter\tallerhand\tlasterhafte\tNeigungen\tangenommen\thatte.\tEr\twar\nboshaft,\thalsstarrig\tund\tungehorsam\tgegen\tVater\tund\tMutter.\tKaum\twar\ter\tein\nwenig\therangewachsen,\tso\tkonnten\tihn\tseine\tEltern\tnicht\tmehr\tim\tHause\nzurückhalten.\tEr\tging\tschon\tam\tfrühen\tMorgen\taus\tund\ttat\tden\tganzen\tTag\nnichts,\tals\tauf\tden\tStraßen\tund\töffentlichen\tPlätzen\tmit\tkleinen\tTagdieben\nspielen.\nAls\ter\tein\tHandwerk\terlernen\tsollte,\tnahm\tihn\tsein\tVater\tin\tseine\tBude\tund\nfing\tan,\tihn\tin\tder\tHandhabung\tder\tNadel\tzu\tunterrichten.\tAllein\tweder\tgute\nWor

# Cohere Rerank comparison

In [ ]:
!pip install cohere


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
api_key = COHERE_API_KEY

In [ ]:
import cohere
cohere_client = cohere.Client(api_key)

In [ ]:
cos_docs = [document.page_content for document in cos_documents]
mmr_docs = [document.page_content for document in mmr_documents]

cos_cohere = cohere_client.rerank(model="rerank-multilingual-v2.0", query=QUERY, documents=cos_docs, top_n=RERANK_K)
mmr_cohere = cohere_client.rerank(model="rerank-multilingual-v2.0", query=QUERY, documents=mmr_docs, top_n=RERANK_K)

In [ ]:
cos_cohere.results[0]

RerankResult<document['text']: Alaeddin	und	die	Wunderlampe
M
ustafa	war	der	Name	eines	Schneiders,	der	in	einer	sehr	reichen	und	großen
Hauptstadt	Chinas	lebte.	Dieser	Mustafa	war	sehr	arm,	und	seine	Arbeit	warf
kaum	so	viel	ab,	daß	er,	seine	Frau	und	ein	Sohn	davon	leben	konnten.
Die	Erziehung	dieses	Sohnes,	welcher	Alaeddin	hieß,	war	sehr	vernachlässigt
worden,	so	daß	er	allerhand	lasterhafte	Neigungen	angenommen	hatte.	Er	war
boshaft,	halsstarrig	und	ungehorsam	gegen	Vater	und	Mutter.	Kaum	war	er	ein
wenig	herangewachsen,	so	konnten	ihn	seine	Eltern	nicht	mehr	im	Hause
zurückhalten.	Er	ging	schon	am	frühen	Morgen	aus	und	tat	den	ganzen	Tag
nichts,	als	auf	den	Straßen	und	öffentlichen	Plätzen	mit	kleinen	Tagdieben
spielen.
Als	er	ein	Handwerk	erlernen	sollte,	nahm	ihn	sein	Vater	in	seine	Bude	und
fing	an,	ihn	in	der	Handhabung	der	Nadel	zu	unterrichten.	Allein	weder	gute
Worte	noch	Drohungen	vermochten	den	flatterhaften	Sinn	des	Sohnes	zu	fesseln.
Kaum	hatte	Mustafa	ihm	den	Rücken	g

# Compare top 10 results
Now we compare how any of the cohere top 10 results are part of the top 10 results of the open source cross encoder model

In [ ]:
cohere_idx = []

for doc in cos_cohere.results:
    cohere_idx.append(doc.index)

cohere_idx

[0, 61, 18, 34, 39]

In [ ]:
i = 0
cos_idx = []

for idx in cos_similarity_scores_sorted[-RERANK_K:]:
    cos_idx.append(idx)
    i += 1
    if i == 10:
        break
cos_idx

[56, 34, 65, 11, 0]

In [ ]:
i = 0
mmr_idx = []

for idx in mmr_similarity_scores_sorted[-RERANK_K:]:
    mmr_idx.append(idx)
    i += 1
    if i == 10:
        break
mmr_idx

[67, 51, 35, 12, 0]

In [ ]:
def get_hitrate(source_list, target_list):
    hit = 0
    total_len = len(target_list)
    
    for entry in source_list:
        if entry in target_list:
            hit += 1
            
    return hit / total_len

In [ ]:
print("Cos similarity rerank cohere hit rate: ", get_hitrate(cos_idx, cohere_idx))

Cos similarity rerank cohere hit rate:  0.4


In [ ]:
print("Cos similarity rerank cohere hit rate: ", get_hitrate(mmr_idx, cohere_idx))

Cos similarity rerank cohere hit rate:  0.2


In [ ]:
cos_documents[61]

Document(page_content='gegenüber\tsetze,\tdamit\tich,\twenn\tmir\tauch\tdas\tVergnügen\tversagt\tist,\tihn\npersönlich\tals\tVater\teiner\tmir\tso\tteuren\tFamilie\tzu\tsehen,\tmir\twenigstens\neinbilden\tkann,\ter\tsitze\tnoch\tdort.«\tAlaeddins\tMutter\tdrang\tnun\tnicht\tweiter\tin\nihn\tund\tließ\tihn\tPlatz\tnehmen,\two\ter\tLust\thatte.\nAls\tder\tafrikanische\tZauberer\tsich\tda\tgesetzt\thatte,\two\tes\tihm\tam\tbesten\nbehagte,\tfing\ter\tein\tGespräch\tmit\tAlaeddins\tMutter\tan:\t»Meine\tliebe\nSchwester,«\tsagte\ter,\t»wundere\tdich\tnicht,\tdaß\tdu\twährend\tder\tganzen\tZeit,\tda\ndu\tmit\tmeinem\tBruder\tMustafa\tverheiratet\twarst,\tmich\tnie\tgesehen\thast.\tEs\tsind\nschon\tvierzig\tJahre,\tdaß\tich\tdieses\tLand\tverlassen\thabe.\tSeitdem\thabe\tich\nReisen\tnach\tIndien,\tPersien,\tArabien,\tSyrien\tund\tÄgypten\tgemacht,\tmich\tin\tden\nschönsten\tStädten\tdieser\tLänder\taufgehalten\tund\tbin\tdann\tnach\tAfrika\ngegangen,\two\tich\teinen\tlängeren\tAufenthalt\tna